In [3]:
import pandas as pd
import numpy as np
import re

# Contexto

Agregar el tema de cada cuanto se va hacer

El paso a paso para el nuevo proceso de validación de RFC de los repartidores se divide en los siguientes pasos:
1. **Raw data:** Descargar raw data del sistema
2. **Data cleaning:** Limpiar los campos donde el segundo apellido sea inconsistente
3. **SAT RFC Validator:** Subir al RFC validator con los nombres limpios
4. **Respuestas SAT merge:** Tomar el resultado del RFC validator, y volver a colacar los campos cambiados en el paso 2 en el resultado del RFC validator.

**Nota:** Asegurarse que todo el tiempo el formato sea UTF-8 además no escribir valores nulos en el resultado final

<img src="./assets/estructura_rfc.png" alt="drawing" style="width:500px;"/>

# Step by step
---

In [4]:
FILE="20240813001"

### 1. Raw Data


Para extraer los datos de Gattaran siga el siguiente tutorial:

<img src="./assets/gattaran1.png" alt="drawing" style="width:400px;"/>
<br>
<img src="./assets/gattaran2.png" alt="drawing" style="width:400px;"/>
<br>
<img src="./assets/gattaran3.png" alt="drawing" style="width:400px;"/>
<br>
<img src="./assets/gattaran4.png" alt="drawing" style="width:250px;"/>



Se extrae la siguiente información del sistema [Link](https://gattaran.didi-food.com/v2/gtr_delivery-op/rider/taxInfoCheck), la tabla que se obtiene tiene las siguientes columnas:
- **ID**: identificador del registro
- **RFC Code**: codigo RFC registrado en la aplicación
- **Full name**: Nombre completo del repartidor
- **Postal code**: Codigo postal del repartidor registrado en la aplicación


In [13]:
gattaran = pd.read_csv('./input/raw/'+FILE+".txt", sep='|', header=None, dtype=str)
#Renombrar las columnas
gattaran.rename(columns={
  0:"ID",
  1:"RFC Code",
  2:"Full name",
  3:"Postal code"
}, inplace=True)
#Sabemos que la columna 0 es un autoincremental que no se puede repetir por lo tanto lo tomaremos como ID
gattaran.set_index("ID", inplace=True)
gattaran

,RFC Code,Full name,Postal code
ID,,,
1,CAMB9604095M4,BOGART ALBERTO CARREÑO MENDOZA,07207
2,RUJC8505299I4,CHRISTIAN SAUL RUIZ JIMENEZ,66000
3,HESD850320BB1,JOSE DAVID HERNANDEZ SANCHEZ,04260
4,VEBS990220252,SALVADOR VEGA BARRAGAN,44350
5,LEJO900922BL5,ORLANDO RAUL DE LEON JIMENEZ,07500
...,...,...,...
2496,AUBD000729J55,DONOVAN ALFONSO ANGUIANO BECERRIL,06350
2497,MABJ671111I85,MA. DE JESUS MARTINEZ BENITEZ,40894
2498,HEME980725SU7,EQUER AZGAD HERNANDEZ MARTINEZ,09000


### 2. Data cleaning

Encontrar unicamente aquellos nombres que en cualquier posición (primer nombre, segundo nombre, primer apellido, segundo apellido) contengan un caracter especial distinto a letras, o sean de la forma X, XX, ..., estos valores se guardan en una matriz `results` que contiene cada componente del nombre por columna, pueden ser más de 4 columnas debido a espacios en blanco. 

In [14]:
matrix_name=gattaran['Full name'].str.split(" ", expand=True)
pattern = r'([^a-zA-ZñÑ]$|^X+$)'
results=matrix_name[matrix_name.apply(lambda row: row.astype(str).str.contains(pattern, regex=True).any(), axis=1)]
results


C:\Users\DiDi\AppData\Local\Temp\ipykernel_18980\232940726.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  results=matrix_name[matrix_name.apply(lambda row: row.astype(str).str.contains(pattern, regex=True).any(), axis=1)]
C:\Users\DiDi\AppData\Local\Temp\ipykernel_18980\232940726.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  results=matrix_name[matrix_name.apply(lambda row: row.astype(str).str.contains(pattern, regex=True).any(), axis=1)]
C:\Users\DiDi\AppData\Local\Temp\ipykernel_18980\232940726.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  results=matrix_name[matrix_name.apply(lambda row: row.astype(str).str.contains(pattern, regex=True).any(), axis=1)]
C:\Users\DiDi\AppData\Local\Temp\ipykernel_18980\232

,0,1,2,3,4,5,6
ID,,,,,,,
194,CARLOS,GARCIA,XXX,None,None,None,None
303,CARLOS,DE,LEON,X,None,None,None
1043,FERNANDO,LOMELI,JR,X,None,None,None
1338,MATIAS,EZEQUIEL,BOSSI,X,None,None,None
1374,LUIS,FERNANDO,XXX,CABALLERO,None,None,None
1412,JESUS,EMANUEL,RODRIGUEZ,.,None,None,None
1562,ANA,KAREN,X,CRUZ,None,None,None
1763,SARAHI,X,CASTELLANOS,None,None,None,None
1788,ERIK,CONTRERAS,.,None,None,None,None


Habiendo identificado los casos se procede a realizar una limpieza de estos valores por cadenas vacias, además se crea la columna donde estará el nombre completo limpio.

In [15]:
# Esta porcion es meramente visual para reconocer las filas y los valores que estan cambiado
pattern = r'([^a-zA-ZñÑ]$|^X+$)|None'
results.apply(lambda row: row.astype(str).str.replace(pattern,'', regex=True), axis=1)

,0,1,2,3,4,5,6
ID,,,,,,,
194,CARLOS,GARCIA,,,,,
303,CARLOS,DE,LEON,,,,
1043,FERNANDO,LOMELI,JR,,,,
1338,MATIAS,EZEQUIEL,BOSSI,,,,
1374,LUIS,FERNANDO,,CABALLERO,,,
1412,JESUS,EMANUEL,RODRIGUEZ,,,,
1562,ANA,KAREN,,CRUZ,,,
1763,SARAHI,,CASTELLANOS,,,,
1788,ERIK,CONTRERAS,,,,,


In [16]:
# Función para unir solo las columnas no vacías
def join_non_empty(row):
    # Filtrar las columnas no vacías
    non_empty_values = [value for value in row if value]
    # Unir los valores con ' '
    return ' '.join(non_empty_values)
pattern = r'([^a-zA-ZñÑ]$|^X+$)|None'

results=results.apply(lambda row: row.astype(str).str.replace(pattern,'', regex=True), axis=1)
results['Full name'] = results.apply(join_non_empty, axis=1)
results['Full name'] = results['Full name'].str.rstrip()
results

,0,1,2,3,4,5,6,Full name
ID,,,,,,,,
194,CARLOS,GARCIA,,,,,,CARLOS GARCIA
303,CARLOS,DE,LEON,,,,,CARLOS DE LEON
1043,FERNANDO,LOMELI,JR,,,,,FERNANDO LOMELI JR
1338,MATIAS,EZEQUIEL,BOSSI,,,,,MATIAS EZEQUIEL BOSSI
1374,LUIS,FERNANDO,,CABALLERO,,,,LUIS FERNANDO CABALLERO
1412,JESUS,EMANUEL,RODRIGUEZ,,,,,JESUS EMANUEL RODRIGUEZ
1562,ANA,KAREN,,CRUZ,,,,ANA KAREN CRUZ
1763,SARAHI,,CASTELLANOS,,,,,SARAHI CASTELLANOS
1788,ERIK,CONTRERAS,,,,,,ERIK CONTRERAS


In [17]:
results=results["Full name"]
results

ID
194                    CARLOS GARCIA
303                   CARLOS DE LEON
1043              FERNANDO LOMELI JR
1338           MATIAS EZEQUIEL BOSSI
1374         LUIS FERNANDO CABALLERO
1412         JESUS EMANUEL RODRIGUEZ
1562                  ANA KAREN CRUZ
1763              SARAHI CASTELLANOS
1788                  ERIK CONTRERAS
1920                   EVA ESPERANZA
1939          ANTONIO GONZALEZ LOPEZ
2461           JOSE CARLOS MORENO JR
2497    MA DE JESUS MARTINEZ BENITEZ
Name: Full name, dtype: object

Por último volvemos a reescriber las filas donde hubo algún cambio en la estructura del nombre en nuestra dataframe inicial `gattaran`.

In [18]:
gattaran['Full name'] = gattaran['Full name'].where(~gattaran.index.isin(results.index), 
                                  gattaran.index.map(results))
gattaran

,RFC Code,Full name,Postal code
ID,,,
1,CAMB9604095M4,BOGART ALBERTO CARREÑO MENDOZA,07207
2,RUJC8505299I4,CHRISTIAN SAUL RUIZ JIMENEZ,66000
3,HESD850320BB1,JOSE DAVID HERNANDEZ SANCHEZ,04260
4,VEBS990220252,SALVADOR VEGA BARRAGAN,44350
5,LEJO900922BL5,ORLANDO RAUL DE LEON JIMENEZ,07500
...,...,...,...
2496,AUBD000729J55,DONOVAN ALFONSO ANGUIANO BECERRIL,06350
2497,MABJ671111I85,MA DE JESUS MARTINEZ BENITEZ,40894
2498,HEME980725SU7,EQUER AZGAD HERNANDEZ MARTINEZ,09000


In [19]:
empty_row=pd.DataFrame({},index=[0])
final_clean=pd.concat([empty_row,gattaran])
final_clean

,RFC Code,Full name,Postal code
0,NaN,NaN,NaN
1,CAMB9604095M4,BOGART ALBERTO CARREÑO MENDOZA,07207
2,RUJC8505299I4,CHRISTIAN SAUL RUIZ JIMENEZ,66000
3,HESD850320BB1,JOSE DAVID HERNANDEZ SANCHEZ,04260
4,VEBS990220252,SALVADOR VEGA BARRAGAN,44350
...,...,...,...
2496,AUBD000729J55,DONOVAN ALFONSO ANGUIANO BECERRIL,06350
2497,MABJ671111I85,MA DE JESUS MARTINEZ BENITEZ,40894
2498,HEME980725SU7,EQUER AZGAD HERNANDEZ MARTINEZ,09000
2499,GABO021130BF8,OSWALDO GARCIA BECERRA,45653


In [21]:
final_clean.to_csv("./output/clean/"+FILE+"_clean.txt",sep="|", header=None)
"""
with open("./output/clean/"+FILE+"_clean.txt", 'r') as file:
    contenido = file.read()

contenido_limpio = re.sub(r'\|{2,}', '|', contenido)

with open("./output/clean/"+FILE+"_clean.txt", 'w') as file:
    file.write(contenido_limpio)
"""

<>:2: SyntaxWarning: invalid escape sequence '\|'
<>:2: SyntaxWarning: invalid escape sequence '\|'
C:\Users\DiDi\AppData\Local\Temp\ipykernel_18980\1683810296.py:2: SyntaxWarning: invalid escape sequence '\|'
  """


'\nwith open("./output/clean/"+FILE+"_clean.txt", \'r\') as file:\n    contenido = file.read()\n\ncontenido_limpio = re.sub(r\'\\|{2,}\', \'|\', contenido)\n\nwith open("./output/clean/"+FILE+"_clean.txt", \'w\') as file:\n    file.write(contenido_limpio)\n'

In [553]:
INTENTIONAL_ERROR

NameError: name 'INTENTIONAL_ERROR' is not defined

### 3. SAT RFC Validator

Suba al [RFC Validator](https://agsc.siat.sat.gob.mx/PTSC/ValidaRFC/) el archivo generado en la carpeta `./output/clean`, guarde el resultado de la consulta con en la carpeta `./input/sat` con el nombre **RESPUESTA_SAT_RFC_FILE.txt** donde FILE corresponde al serial guardado en la variable `FILE`

### 4. Respuesta SAT merge

Los resultados posibles del SAT son los siguientes.

<img src="./assets/posible_resultados.png" alt="drawing" style="width:500px;"/>

Cuando el resultado es ***Estructura del RFC incorrecta*** el resultado aparece en la columna dos remplazando el valor del nombre y el restante de valores.

In [19]:
sat = pd.read_csv('./input/sat/RESPUESTA_SAT_RFC_'+FILE+'.txt', sep='|', header=None, encoding='latin-1',  dtype=str)
sat.rename(columns={
    0:"ID",
    1:"RFC Code",
    2:"Full name",
    3:"Postal code",
    4:"Result"}
    ,inplace=True)
sat.set_index("ID", inplace=True)
sat.fillna('',inplace=True)
sat

,RFC Code,Full name,Postal code,Result
ID,,,,
1,MELA750922PV9,ALEJANDRO MELESIO LOPEZ,06010,"RFC válido, y susceptible de recibir facturas"
2,JUAC910613FX7,CESAR SAUL JUAREZ AMEZCUA,63737,"RFC válido, y susceptible de recibir facturas"
3,MEBA000529385,AXEL MENDECUCHIA BALLESTEROS,10810,"RFC válido, y susceptible de recibir facturas"
4,HEMF781213EL6,FRANCISCO NICOLAS HERNANDEZ MARQUEZ,66023,El Código Postal no coincide con el registrado...
5,HEME991216NE9,EFREN ISRAEL HERNANDEZ MONTERO,45605,El Código Postal no coincide con el registrado...
...,...,...,...,...
282,YARF801201LF0,FELIPE YAÑEZ RODRIGUEZ,87086,"RFC válido, y susceptible de recibir facturas"
283,SAMA970829KI0,JOSE ALFREDO SALAZAR MARTINEZ,52105,"RFC válido, y susceptible de recibir facturas"
284,MEDN9408196Y5,NOEMI MENDIETA DE DIOS,14250,"RFC válido, y susceptible de recibir facturas"


Se divide el dataset en dos
- **rfc_incorrecta:** contiene los registros cuyo resultado fue  ***Estructura del RFC incorrecta***
- **filtered:** el cual es el complemento de rfc_incorrecta 

In [22]:
rfc_incorrecta=sat[sat["Full name"]=="Estructura del RFC incorrecta"]
filtered=sat[sat["Full name"]!="Estructura del RFC incorrecta"]
filtered

,RFC Code,Full name,Postal code,Result
ID,,,,
1,MELA750922PV9,ALEJANDRO MELESIO LOPEZ,06010,"RFC válido, y susceptible de recibir facturas"
2,JUAC910613FX7,CESAR SAUL JUAREZ AMEZCUA,63737,"RFC válido, y susceptible de recibir facturas"
3,MEBA000529385,AXEL MENDECUCHIA BALLESTEROS,10810,"RFC válido, y susceptible de recibir facturas"
4,HEMF781213EL6,FRANCISCO NICOLAS HERNANDEZ MARQUEZ,66023,El Código Postal no coincide con el registrado...
5,HEME991216NE9,EFREN ISRAEL HERNANDEZ MONTERO,45605,El Código Postal no coincide con el registrado...
...,...,...,...,...
282,YARF801201LF0,FELIPE YAÑEZ RODRIGUEZ,87086,"RFC válido, y susceptible de recibir facturas"
283,SAMA970829KI0,JOSE ALFREDO SALAZAR MARTINEZ,52105,"RFC válido, y susceptible de recibir facturas"
284,MEDN9408196Y5,NOEMI MENDIETA DE DIOS,14250,"RFC válido, y susceptible de recibir facturas"


Es necesario que los nombres queden de la misma forma en la que estaban en raw data para volver a subir el archivo al sistema.

In [23]:
raw_gattaran=pd.read_csv('./input/raw/'+FILE+".txt", sep='|', header=None, dtype=str)
#Renombrar las columnas
raw_gattaran.rename(columns={
  0:"ID",
  1:"RFC Code",
  2:"Full name",
  3:"Postal code"
}, inplace=True)
#Sabemos que la columna 0 es un autoincremental que no se puede repetir por lo tanto lo tomaremos como ID
raw_gattaran.set_index("ID", inplace=True)
#Tomamos unicamente la columna Full name pues es en esta en la unica que se hicieron cambios
raw_gattaran=raw_gattaran["Full name"]
raw_gattaran

ID
1                   ALEJANDRO MELESIO LOPEZ
2                 CESAR SAUL JUAREZ AMEZCUA
3              AXEL MENDECUCHIA BALLESTEROS
4      FRANCISCO NICOLAS HERNANDEZ MARQUEZ.
5            EFREN ISRAEL HERNANDEZ MONTERO
                       ...                 
282                  FELIPE YAÑEZ RODRIGUEZ
283           JOSE ALFREDO SALAZAR MARTINEZ
284                  NOEMI MENDIETA DE DIOS
285                 JORGE LUIS PECH SARABIA
286                   ARELY PABELLO SALINAS
Name: Full name, Length: 286, dtype: object

Se toman los nombres del archivo originalmente descargado del sistema.

In [24]:
filtered['Full name'] = filtered['Full name'].where(~filtered.index.isin(raw_gattaran.index), 
                                  filtered.index.map(raw_gattaran))

C:\Users\DiDi\AppData\Local\Temp\ipykernel_19272\1572554612.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['Full name'] = filtered['Full name'].where(~filtered.index.isin(raw_gattaran.index),


Se unen los datos resultantes con los resultados anteriormente extraidos cuyo resultado fue `Estructura del RFC incorrecta`, se organizan los valores como venian originalmente. 

In [25]:
final=pd.concat([filtered,rfc_incorrecta])
final.reset_index(inplace=True)
final["ID"]=final["ID"].astype(int)
final=final.sort_values(by="ID")
final.set_index("ID",inplace=True)

In [26]:
final.to_csv("./output/final/RESPUESTA_SAT_RF"+FILE+"_final.txt", header=None, encoding="utf-8",sep="|")
with open("./output/final/RESPUESTA_SAT_RF"+FILE+"_final.txt", 'r', encoding='utf-8') as file:
    contenido = file.read()

contenido_limpio = re.sub(r'\|{2,}', '', contenido)

with open("./output/final/RESPUESTA_SAT_RF"+FILE+"_final.txt", 'w', encoding='utf-8') as file:
    file.write(contenido_limpio)